<a href="https://colab.research.google.com/github/SHAGANTI-NANDHITHA/SKIN-CANCER-PREDICTION/blob/main/Skin_cancer_(CNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [ ]:
!pip install --upgrade kagglehub

import kagglehub

# Download latest version
path = kagglehub.dataset_download("hasnainjaved/melanoma-skin-cancer-dataset-of-10000-images")

In [ ]:
train_dir = "/root/.cache/kagglehub/datasets/hasnainjaved/melanoma-skin-cancer-dataset-of-10000-images/versions/1/melanoma_cancer_dataset/train"
test_dir = "/root/.cache/kagglehub/datasets/hasnainjaved/melanoma-skin-cancer-dataset-of-10000-images/versions/1/melanoma_cancer_dataset/test"


In [ ]:
import os

if os.path.exists(train_dir) and os.path.exists(test_dir):
    print(f"Train directory found: {train_dir}")
    print(f"Test directory found: {test_dir}")
else:
    print("Train or test directory not found! Verify the paths.")


Train directory found: /root/.cache/kagglehub/datasets/hasnainjaved/melanoma-skin-cancer-dataset-of-10000-images/versions/1/melanoma_cancer_dataset/train
Test directory found: /root/.cache/kagglehub/datasets/hasnainjaved/melanoma-skin-cancer-dataset-of-10000-images/versions/1/melanoma_cancer_dataset/test


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 9605 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

for layer in base_model.layers[:-4]:  # Keep the last 4 layers trainable
    layer.trainable = False

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # For binary classification
])

model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 4, 4, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,780,481 (56.38 MB)

 Trainable params: 7,145,217 (27.26 MB)

 Non-trainable params: 7,635,264 (29.13 MB)

In [ ]:
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=10
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


301/301 ━━━━━━━━━━━━━━━━━━━━ 96s 268ms/step - accuracy: 0.8101 - loss: 0.4151 - val_accuracy: 0.9000 - val_loss: 0.2599
Epoch 2/10
301/301 ━━━━━━━━━━━━━━━━━━━━ 124s 246ms/step - accuracy: 0.8845 - loss: 0.2743 - val_accuracy: 0.9000 - val_loss: 0.2190
Epoch 3/10
301/301 ━━━━━━━━━━━━━━━━━━━━ 75s 244ms/step - accuracy: 0.8963 - loss: 0.2455 - val_accuracy: 0.9010 - val_loss: 0.2488
Epoch 4/10
301/301 ━━━━━━━━━━━━━━━━━━━━ 82s 245ms/step - accuracy: 0.9055 - loss: 0.2260 - val_accuracy: 0.9080 - val_loss: 0.2291
Epoch 5/10
301/301 ━━━━━━━━━━━━━━━━━━━━ 74s 242ms/step - accuracy: 0.9091 - loss: 0.2136 - val_accuracy: 0.8990 - val_loss: 0.2399
Epoch 6/10
301/301 ━━━━━━━━━━━━━━━━━━━━ 82s 244ms/step - accuracy: 0.9105 - loss: 0.2164 - val_accuracy: 0.9110 - val_loss: 0.2762
Epoch 7/10
301/301 ━━━━━━━━━━━━━━━━━━━━ 73s 238ms/step - accuracy: 0.9166 - loss: 0.2024 - val_accuracy: 0.9090 - val_loss: 0.2089
Epoch 8/10
301/301 ━━━━━━━━━━━━━━━━━━━━ 82s 240ms/step - accuracy: 0.9211 - loss: 0.1962 - va

In [ ]:
model.save("skin_cancer_model.h5")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
model.save('/content/drive/MyDrive/skin_cancer_model.h5')
